In [1]:
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv



In [2]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [3]:
HG = load_ABCDH_from_file("Data/results_300_he.txt")



In [4]:
hL = hl.hLouvain(HG,hmod_type=hmod.linear, 
                                    delta_it = 0.0001, 
                                    delta_phase = 0.0001, 
                                    random_seed = 8754) 

In [12]:
alphas = []
d = 0
c = 0.6
b = 0.4
for i in range(30):
    alphas.append(1-((1-b)**i)*(1-d))
    
print("alphas", alphas)
    
A, q2, alphas_out = hL.combined_louvain_alphas(alphas = alphas, 
                                                change_mode="communities", community_factor=1/c)
qH = hL.combined_modularity(A, hmod.linear, 1)

print("alphas_out", alphas_out)

print("partition",A)
print("gH =",qH)

alphas [0.0, 0.4, 0.64, 0.784, 0.8704000000000001, 0.9222400000000001, 0.953344, 0.9720064, 0.98320384, 0.989922304, 0.9939533824, 0.99637202944, 0.997823217664, 0.9986939305984, 0.99921635835904, 0.999529815015424, 0.9997178890092544, 0.9998307334055526, 0.9998984400433316, 0.999939064025999, 0.9999634384155993, 0.9999780630493597, 0.9999868378296157, 0.9999921026977695, 0.9999952616186617, 0.999997156971197, 0.9999982941827182, 0.9999989765096309, 0.9999993859057785, 0.9999996315434672]
alphas_out [0.0, 0.4, 0.64, 0.784, 0.8704000000000001, 0.9222400000000001]
partition [{'276', '297', '190', '169', '74', '50', '66', '211', '241', '238', '43', '93', '179', '284', '57', '149', '103', '213'}, {'210', '229', '265', '72', '288', '251', '152', '237', '249', '155', '188', '114', '88', '247', '173', '132', '228', '116', '151', '85', '128', '78', '12', '223', '106', '109', '97', '18', '46', '145', '143', '193', '20'}, {'55', '196', '64', '204', '171', '161', '19', '243', '120', '137', '101',

In [13]:
qH = hL.combined_modularity(A, hmod.linear, 1)
print(qH)
two_section_modularity = hL.combined_modularity(A, hmod.linear, 0)
print(two_section_modularity)

0.5300888277866632
0.45115615643628465


In [10]:
with open("Data/results_300_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
A_gt = [x for x in hmod.dict2part({str(i+1):gt[i] for i in range(len(gt))}) if len(x)>0]
gt_linear = hL.combined_modularity(A_gt, hmod.linear, 1)

print("ground-truth partition")
print(A_gt)
print("qh-gt-strict:",gt_linear)

def getAMI_ARI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ari = [d[str(i+1)] for i in range(len(HG.nodes))]
    return ARI(gt, A4ari), AMI(gt, A4ari)

ground-truth partition
[{'135', '112', '148', '260', '182', '124', '271', '81', '65', '70', '286', '300', '125', '183', '117', '79', '227', '144', '134', '197', '203', '272', '14', '77', '254', '191', '285', '96'}, {'189', '121', '92', '268', '154', '90', '119', '110', '89', '87', '83', '68', '41', '226', '280', '141', '16', '71', '28', '230', '123', '209', '185', '34', '198'}, {'210', '229', '72', '95', '265', '221', '288', '152', '251', '247', '88', '173', '266', '228', '116', '151', '78', '12', '223', '296', '109', '18', '46', '143', '222'}, {'235', '270', '167', '147', '180', '162', '236', '177', '82', '114', '63', '32', '44', '40', '283', '290', '4', '261', '255', '13', '62', '186'}, {'172', '257', '67', '102', '200', '45', '35', '49', '104', '174', '201', '140', '195', '166', '216', '258', '29', '269', '58', '299', '2', '31'}, {'276', '74', '66', '190', '43', '169', '50', '80', '149', '103', '297', '213', '211', '241', '238', '93', '179', '57', '284'}, {'273', '279', '178', '214'

In [14]:
ari, ami = getAMI_ARI(HG,gt,A)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(A))
print("comm-gt =", len(A_gt))

ARI = 0.744035375315832
AMI = 0.8400292934795289
comm = 15
comm-gt = 18
